## **Fichier De Nettoyage des données Extraire De Site Mubawab**

**dans ce notebook je vais essayer de charger et netoyer les données extraire et les enregistrés sous formes de fichier csv au lieu de json**

In [ ]:
# Les importation nécessaires
import pandas as pd 
import json
import os 

In [106]:
# lecture de fichier json
with open("../data/raw/annonces_ventes_mubawab.json", "r", encoding="utf-8") as f:
    annonces = json.load(f)

df_fichier = pd.DataFrame.from_dict(annonces, orient="index").reset_index(drop=True)

'''
j'ai le fichier de JSON sous formes de dictionnaire les colonnes 
sont les annonces et les valeurs sont les attributs de chaque annonce 
c'est faut pour cela ona charge le fichier
'''
#première exploration des données

print(df_fichier.shape)
print(df_fichier.columns)
df_fichier.head()


(17127, 10)
Index(['id', 'ville', 'prix', 'surface', 'quartier', 'type_bien',
       'nb_chambres', 'nb_salle_de_bain', 'url_annonce', 'date_annonce'],
      dtype='object')


,id,ville,prix,surface,quartier,type_bien,nb_chambres,nb_salle_de_bain,url_annonce,date_annonce
0,annonce_1,Casablanca,1 380 000 DH,57 m²,Bourgogne Ouest à\n\t\t\t\t\t\t\tCasablanca,Appartement,1 Chambre,1 Salle de bain,https://www.mubawab.ma/fr/a/8066263/studio-pre...,None
1,annonce_2,Casablanca,1 350 000 DH,54 m²,Casablanca Finance City à\n\t\t\t\t\t\t\tCasab...,Appartement,1 Chambre,1 Salle de bain,https://www.mubawab.ma/fr/a/8247723/studio-pre...,None
2,annonce_3,Casablanca,3 679 110 DH,187 m²,Hay Hassani à\n\t\t\t\t\t\t\tCasablanca,Local commercial,None,1 Salle de bain,https://www.mubawab.ma/fr/pa/8111991/local-com...,None
3,annonce_4,Casablanca,1 450 000 DH,83 m²,Ferme Bretonne (Hay Arraha) à\n\t\t\t\t\t\t\tC...,Appartement,2 Chambres,2 Salles de bains,https://www.mubawab.ma/fr/a/7857210/appartemen...,None
4,annonce_5,Casablanca,2 400 000 DH,84 m²,Casablanca Finance City à\n\t\t\t\t\t\t\tCasab...,Appartement,1 Chambre,1 Salle de bain,https://www.mubawab.ma/fr/a/8278144/vente-stud...,None


Pour commencer la phase de nettoyage les données extraire de site sont des données brute il nécessite une suite des modifications et de nettoyage

In [107]:
# Copie de travail
df_mubawab = df_fichier.copy()

print("AVANT NETTOYAGE")
print(f"Forme: {df_mubawab.shape}")
print(f"Colonnes: {df_mubawab.columns.tolist()}\n")

# Conversion en string pour les colonnes à nettoyer
cols_str = [
    "id", "ville", "quartier", "prix", "surface",
    "nb_chambres", "nb_salle_de_bain", "url_annonce", "type_bien"
]
for col in cols_str:
    if col in df_mubawab.columns:
        df_mubawab[col] = df_mubawab[col].astype(str)

# ===== PRIX =====
if "prix" in df_mubawab.columns:
    df_mubawab["prix"] = (
        df_mubawab["prix"]
        .str.replace("DH", "", regex=False)
        .str.replace("EUR", "", regex=False)
        .str.replace("\xa0", "", regex=False)
        .str.replace(" ", "", regex=False)
        .str.strip()
    )
    df_mubawab["prix"] = pd.to_numeric(df_mubawab["prix"], errors="coerce").astype(float)

# ===== SURFACE =====
if "surface" in df_mubawab.columns:
    df_mubawab["surface"] = (
        df_mubawab["surface"]
        .str.replace("m²", "", regex=False)
        .str.replace("m2", "", regex=False)
        .str.replace("\xa0", "", regex=False)
        .str.replace(" ", "", regex=False)
        .str.strip()
    )
    df_mubawab["surface"] = pd.to_numeric(df_mubawab["surface"], errors="coerce").astype(float)

# ===== CHAMBRES =====
if "nb_chambres" in df_mubawab.columns:
    df_mubawab["nb_chambres"] = (
        df_mubawab["nb_chambres"]
        .str.extract(r"(\d+)", expand=False)
    )
    df_mubawab["nb_chambres"] = pd.to_numeric(
        df_mubawab["nb_chambres"], errors="coerce"
    ).astype('Int64')

# ===== SALLES DE BAIN =====
if "nb_salle_de_bain" in df_mubawab.columns:
    df_mubawab["nb_salle_de_bain"] = (
        df_mubawab["nb_salle_de_bain"]
        .str.extract(r"(\d+)", expand=False)
    )
    df_mubawab["nb_salle_de_bain"] = pd.to_numeric(
        df_mubawab["nb_salle_de_bain"], errors="coerce"
    ).astype('Int64')

# ===== QUARTIER =====
if "quartier" in df_mubawab.columns:
    df_mubawab["quartier"] = (
        df_mubawab["quartier"]
        .str.replace("\n", " ", regex=False)
        .str.replace("\t", " ", regex=False)
        .str.replace(r"\s+à\s+.*$", "", regex=True)
        .str.strip()
    )

# ===== VILLE =====
if "ville" in df_mubawab.columns:
    df_mubawab["ville"] = df_mubawab["ville"].str.strip().str.title()

# ===== TYPE_BIEN =====
if "type_bien" in df_mubawab.columns:
    df_mubawab["type_bien"] = df_mubawab["type_bien"].str.strip().str.title()

# ===== SUPPRESSION DES COLONNES INUTILES =====
df_mubawab.drop(
    columns=["date_annonce"],
    inplace=True,
    errors="ignore"
)

print("APRES NETTOYAGE - TYPES DE DONNÉES")
print(df_mubawab.dtypes)


AVANT NETTOYAGE
Forme: (17127, 10)
Colonnes: ['id', 'ville', 'prix', 'surface', 'quartier', 'type_bien', 'nb_chambres', 'nb_salle_de_bain', 'url_annonce', 'date_annonce']

APRES NETTOYAGE - TYPES DE DONNÉES
id                   object
ville                object
prix                float64
surface             float64
quartier             object
type_bien            object
nb_chambres           Int64
nb_salle_de_bain      Int64
url_annonce          object
dtype: object


In [108]:
# apercu des données nettoyées
df_mubawab.head()


,id,ville,prix,surface,quartier,type_bien,nb_chambres,nb_salle_de_bain,url_annonce
0,annonce_1,Casablanca,1380000.0,57.0,Bourgogne Ouest,Appartement,1,1,https://www.mubawab.ma/fr/a/8066263/studio-pre...
1,annonce_2,Casablanca,1350000.0,54.0,Casablanca Finance City,Appartement,1,1,https://www.mubawab.ma/fr/a/8247723/studio-pre...
2,annonce_3,Casablanca,3679110.0,187.0,Hay Hassani,Local Commercial,<NA>,1,https://www.mubawab.ma/fr/pa/8111991/local-com...
3,annonce_4,Casablanca,1450000.0,83.0,Ferme Bretonne (Hay Arraha),Appartement,2,2,https://www.mubawab.ma/fr/a/7857210/appartemen...
4,annonce_5,Casablanca,2400000.0,84.0,Casablanca Finance City,Appartement,1,1,https://www.mubawab.ma/fr/a/8278144/vente-stud...


**Passons à la supression et le remplissage des valeurs manquantes selon une logique**
----
**logique de traitement**


    . extraire les types des biens et garde seulement les types désirées 
    . pour les salles de bain que nous avons none ou nan en met 1 par défaut (il n'existe pas une maison ou appartement ou villa sans salle de bain)
    . pour nb_chambres est none ou nan nous supprimons ces lignes 
    . quartie vide nous le remplacons par nom de ville 
    . prix none ou na nous le remplacons par la moyenne des prix par ville
    . surface -> calcul de prix par m2 et voire la colonne prix et selon le prix calcul la surface 

In [109]:
# voyans les types des biens qu'on a dans le dataset
valeurs_uniques = df_mubawab['type_bien'].unique()
print(valeurs_uniques)

['Appartement' 'Local Commercial' 'Villa' 'None' 'Riad' 'Bureau' 'Maison'
 'Logement' 'Ferme']


In [110]:
# supprimons les doublons 
df_mubawab.drop_duplicates(inplace=True)
df_mubawab.shape
# suppression les lignes avec types de biens non désirés
df_mubawab = df_mubawab[df_mubawab['type_bien'].notna()]        # supprime les NaN
df_mubawab = df_mubawab[df_mubawab['type_bien'] != 'None']      # # supprime les None
# On garde uniquement les types utiles
types_utiles = ['Appartement', 'Maison', 'Villa', 'Riad']
df_mubawab = df_mubawab[df_mubawab['type_bien'].isin(types_utiles)]


In [112]:
# NB_SALLE_DE_BAIN : remplacer None ou NaN par 1
df_mubawab['nb_salle_de_bain'] = df_mubawab['nb_salle_de_bain'].fillna(1)

# NB_CHAMBRES : supprimer les lignes où c'est None ou NaN
df_mubawab = df_mubawab[df_mubawab['nb_chambres'].notna()]

# QUARTIER : remplacer les valeurs vides par le nom de la ville
df_mubawab['quartier'] = df_mubawab['quartier'].fillna(df_mubawab['ville'])
df_mubawab.loc[df_mubawab['quartier'] == '<NA>', 'quartier'] = df_mubawab['ville']

# PRIX : remplacer les valeurs None/NaN par la moyenne des prix par ville
df_mubawab['prix'] = df_mubawab.groupby('ville')['prix'].transform(lambda x: x.fillna(x.mean()))

# Calculer le prix au m² pour les lignes où surface existe
df_mubawab['prix_m2'] = df_mubawab['prix'] / df_mubawab['surface']

# Calculer le prix moyen au m² par ville
prix_m2_par_ville = df_mubawab.groupby('ville')['prix_m2'].mean().to_dict()

# Remplir les surfaces manquantes
for index, row in df_mubawab.iterrows():
    if pd.isna(row['surface']):
        ville = row['ville']
        df_mubawab.at[index, 'surface'] = row['prix'] / prix_m2_par_ville[ville]

# Supprimer la colonne temporaire prix_m2
df_mubawab.drop(columns=['prix_m2'], inplace=True)

# Vérification
print(df_mubawab.isna().sum())
df_mubawab.head()
df_mubawab.shape

id                  0
ville               0
prix                0
surface             0
quartier            0
type_bien           0
nb_chambres         0
nb_salle_de_bain    0
url_annonce         0
dtype: int64


(14746, 9)

In [113]:
# enregistrer le dataframe nettoyé sous forme de fichier CSV
chemin_dossier = '../data/clean_data/'
if not os.path.exists(chemin_dossier):
    os.makedirs(chemin_dossier)

# Exporter le DataFrame nettoyé en CSV
chemin_fichier = os.path.join(chemin_dossier, 'annonces_nettoyees_mubawab.csv')
df_mubawab.to_csv(chemin_fichier, index=False, encoding='utf-8')